In [4]:
import os
import xarray as xr

# === Paths ===
input_folder = r"Data - Modelling\Annual_DB\2. Reg_Cropped\Rainfall"  # Change for each variable
output_folder = r"Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall"
os.makedirs(output_folder, exist_ok=True)

# === List .nc files ===
nc_files = [f for f in os.listdir(input_folder) if f.endswith(".nc")]

# === Process each file ===
for nc_file in nc_files:
    nc_path = os.path.join(input_folder, nc_file)
    print(f"\n📂 Processing: {nc_file}")

    try:
        # === Load the QLD-cropped file ===
        ds = xr.open_dataset(nc_path)
        
        # === Ensure lat, lon, region_code are preserved ===
        coords_to_keep = ["lat", "lon", "region_code"] if "region_code" in ds else ["lat", "lon"]
        numeric_vars = [var for var in ds.data_vars if ds[var].dtype in ["float32", "float64", "int32", "int64"]]

        # === Convert numeric variables to monthly average ===
        ds_monthly = ds[numeric_vars].resample(time="MS").mean()  # 'MS' is monthly start
        
        # === Preserve non-averaged coordinates and region_code ===
        for coord in coords_to_keep:
            if coord in ds:
                ds_monthly[coord] = ds[coord]

        # === Save monthly-averaged file ===
        output_path = os.path.join(output_folder, f"Monthly_{nc_file}")
        ds_monthly.to_netcdf(output_path)
        print(f"✅ Saved monthly average file: {output_path}")

    except Exception as e:
        print(f"❌ Failed for {nc_file}: {e}")



📂 Processing: BRB_cropped_2000.monthly_rain.nc
✅ Saved monthly average file: Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall\Monthly_BRB_cropped_2000.monthly_rain.nc

📂 Processing: BRB_cropped_2001.monthly_rain.nc
✅ Saved monthly average file: Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall\Monthly_BRB_cropped_2001.monthly_rain.nc

📂 Processing: BRB_cropped_2002.monthly_rain.nc
✅ Saved monthly average file: Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall\Monthly_BRB_cropped_2002.monthly_rain.nc

📂 Processing: BRB_cropped_2003.monthly_rain.nc
✅ Saved monthly average file: Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall\Monthly_BRB_cropped_2003.monthly_rain.nc

📂 Processing: BRB_cropped_2004.monthly_rain.nc
✅ Saved monthly average file: Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall\Monthly_BRB_cropped_2004.monthly_rain.nc

📂 Processing: BRB_cropped_2005.monthly_rain.nc
✅ Saved monthly average file: Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall\Monthly_BRB_cropped_2005.monthly_

In [6]:
ds = xr.open_dataset("Data - Modelling\Monthly_DB\Reg_Cropped\Max Temp\Monthly_CHC_cropped_2010.max_temp.nc")
print(ds)

<xarray.Dataset> Size: 1MB
Dimensions:      (time: 12, lat: 130, lon: 117)
Coordinates:
  * lat          (lat) float64 1kB -28.95 -28.9 -28.85 ... -22.6 -22.55 -22.5
  * lon          (lon) float64 936B 138.0 138.1 138.1 ... 143.7 143.8 143.8
  * time         (time) datetime64[ns] 96B 2010-01-01 2010-02-01 ... 2010-12-01
Data variables:
    spatial_ref  (time) float64 96B ...
    max_temp     (time, lat, lon) float64 1MB ...
Attributes:
    department:               Department of Environment and Science
    department_short:         DES
    copyright:                Copyright - the State of Queensland Department ...
    site_url:                 http://www.longpaddock.qld.gov.au
    institution:              Queensland Government, Department of Environmen...
    raster_source:            Gridded surface was created by interpolating ob...
    raster_source_additions:  and other suppliers (see the SILO webpage for d...
    metadata_url:             http://qldspatial.information.qld.gov.au

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Jspch\AppData\Local\Temp\ipykernel_19848\880296759.py:1: SyntaxWarning: invalid escape sequence '\M'
  ds = xr.open_dataset("Data - Modelling\Monthly_DB\Reg_Cropped\Max Temp\Monthly_CHC_cropped_2010.max_temp.nc")


SPI Generation from Rainfall data

In [ ]:
import xarray as xr
import numpy as np
import os
from scipy.stats import gamma, norm
from tqdm import tqdm

# === Paths ===
input_dir = "Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall"
output_dir = "Data - Modelling\Monthly_DB\Reg_Cropped\SPI"
os.makedirs(output_dir, exist_ok=True)

def calculate_spi_1(rain_ts):
    """Calculate SPI-1 for a 1D time series of rainfall."""
    if np.isnan(rain_ts).any() or np.all(rain_ts <= 0):
        return np.full_like(rain_ts, np.nan)

    rain_ts = np.where(rain_ts <= 0, 0.1, rain_ts)
    try:
        shape, loc, scale = gamma.fit(rain_ts, floc=0)
        cdf_vals = gamma.cdf(rain_ts, shape, loc=loc, scale=scale)
        spi_vals = norm.ppf(cdf_vals)
        return spi_vals
    except:
        return np.full_like(rain_ts, np.nan)

# === Process Each File ===
for file in tqdm(sorted(os.listdir(input_dir))):
    if not file.endswith(".nc"):
        continue

    print(f"\n📂 Processing: {file}")
    ds = xr.open_dataset(os.path.join(input_dir, file))
    rain = ds['monthly_rain'].values

    # === Preserve coordinates and region_code ===
    time, lat, lon = rain.shape
    spi_vals = np.full((time, lat, lon), np.nan)
    region_code = ds['region_code'] if 'region_code' in ds else None

    for i in range(lat):
        for j in range(lon):
            rain_ts = rain[:, i, j]
            spi = calculate_spi_1(rain_ts)
            spi_vals[:, i, j] = spi

    # === Save SPI file while preserving coordinates ===
    spi_ds = xr.Dataset({
        'spi_1': (['time', 'lat', 'lon'], spi_vals),
    }, coords={
        'time': ds.time,
        'lat': ds.lat,
        'lon': ds.lon
    })

    # === Preserve region_code if it exists ===
    if region_code is not None:
        spi_ds['region_code'] = ds['region_code']

    # === Save the SPI NetCDF file using a unique filename ===
    output_filename = f"SPI_{file}"
    out_path = os.path.join(output_dir, output_filename)
    spi_ds.to_netcdf(out_path)
    print(f"✅ Saved: {out_path}")


<>:8: SyntaxWarning: invalid escape sequence '\M'
<>:9: SyntaxWarning: invalid escape sequence '\M'
<>:8: SyntaxWarning: invalid escape sequence '\M'
<>:9: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Jspch\AppData\Local\Temp\ipykernel_19848\1981516166.py:8: SyntaxWarning: invalid escape sequence '\M'
  input_dir = "Data - Modelling\Monthly_DB\Reg_Cropped\Rainfall"
C:\Users\Jspch\AppData\Local\Temp\ipykernel_19848\1981516166.py:9: SyntaxWarning: invalid escape sequence '\M'
  output_dir = "Data - Modelling\Monthly_DB\Reg_Cropped\SPI"
  0%|          | 0/325 [00:00<?, ?it/s]


📂 Processing: Monthly_BRB_cropped_2000.monthly_rain.nc


  0%|          | 1/325 [00:02<14:29,  2.68s/it]

✅ Saved: Data - Modelling\Monthly_DB\Reg_Cropped\SPI\SPI_Monthly_BRB_cropped_2000.monthly_rain.nc

📂 Processing: Monthly_BRB_cropped_2001.monthly_rain.nc


  1%|          | 2/325 [00:05<14:42,  2.73s/it]

...

✅ Saved: Data - Modelling\Monthly_DB\Reg_Cropped\SPI\SPI_Monthly_WET_cropped_2020.monthly_rain.nc

📂 Processing: Monthly_WET_cropped_2021.monthly_rain.nc
✅ Saved: Data - Modelling\Monthly_DB\Reg_Cropped\SPI\SPI_Monthly_WET_cropped_2021.monthly_rain.nc

📂 Processing: Monthly_WET_cropped_2022.monthly_rain.nc


 99%|█████████▉| 323/325 [05:36<00:00,  5.30it/s]

✅ Saved: Data - Modelling\Monthly_DB\Reg_Cropped\SPI\SPI_Monthly_WET_cropped_2022.monthly_rain.nc

📂 Processing: Monthly_WET_cropped_2023.monthly_rain.nc


100%|██████████| 325/325 [05:37<00:00,  1.04s/it]

✅ Saved: Data - Modelling\Monthly_DB\Reg_Cropped\SPI\SPI_Monthly_WET_cropped_2023.monthly_rain.nc

📂 Processing: Monthly_WET_cropped_2024.monthly_rain.nc
✅ Saved: Data - Modelling\Monthly_DB\Reg_Cropped\SPI\SPI_Monthly_WET_cropped_2024.monthly_rain.nc


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd

# === Load the shapefile for Queensland regions ===
shapefile_path = r'Data\QLD Bio Regions\QLD_BioRegion_Shape_Files\Biogeographic_Regions.shp'
qld_shape = gpd.read_file(shapefile_path).to_crs("EPSG:4326")

# === Open the monthly max temperature data for QLD ===
ds = xr.open_dataset(r'Data - Modelling\Region_Monthly_DB\1. Reg_Cropped_Separated\CYP\Rainfall\Monthly_CYP_cropped_2000.monthly_rain.nc')
monthly_rain = ds['monthly_rain']  # Adjust variable name as needed

# === Create figure and axes for the plot ===
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(14, 12))
axes = axes.flatten()

# === Plot each month with its own colorbar ===
for i, month in enumerate(monthly_rain.time):
    ax = axes[i]
    data = monthly_rain.sel(time=month)
    
    # Plot with individual colorbar
    data.plot(ax=ax, cmap='plasma', add_colorbar=True)
    
    # Overlay region boundaries
    qld_shape.boundary.plot(ax=ax, color='black', linewidth=0.5, alpha=0.7)
    
    # Set the title
    ax.set_title(month.dt.strftime('%B').item())
    ax.set_xlabel('')
    ax.set_ylabel('')

# === Remove unused axes if any ===
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

# === Final layout ===
plt.tight_layout()
plt.show()
